<a href="https://colab.research.google.com/github/PremChand-Neela/Elderly-fall-detection/blob/main/finalcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import pandas as pd
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# ----------------------
# CONFIG
# ----------------------
WINDOW_SIZE = 50
SENSOR_COLS = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
RANDOM_STATE = 42
EPOCHS = 50
BATCH_SIZE = 64

# ----------------------
# STEP 1: DOWNLOAD DATASET FROM KAGGLE
# ----------------------
print("Downloading dataset from Kaggle...")
dataset_path = kagglehub.dataset_download("ziya07/elderly-fall-detection-iot-dataset")

print("Dataset downloaded at:", dataset_path)

# The folder contains multiple CSV files. Find the actual data file.
csv_files = [f for f in os.listdir(dataset_path) if f.endswith(".csv")]

if not csv_files:
    raise FileNotFoundError("No CSV files found inside downloaded Kaggle dataset folder!")

csv_path = os.path.join(dataset_path, csv_files[0])
print("Using data file:", csv_path)

# ----------------------
# STEP 2: LOAD CSV DATA
# ----------------------
df = pd.read_csv(csv_path)
df = df.dropna().reset_index(drop=True)

# Check sensor columns
for c in SENSOR_COLS:
    if c not in df.columns:
        raise ValueError(f"Column '{c}' missing. Check dataset column names.")

if "label" not in df.columns:
    raise ValueError("Dataset must contain a label column named 'label'.")

X_all = df[SENSOR_COLS].values.astype(np.float32)
y_raw = df["label"].values

# ----------------------
# STEP 3: ENCODE LABELS
# ----------------------
le = LabelEncoder()
y_int = le.fit_transform(y_raw)
num_classes = len(le.classes_)

print("Detected Classes:", list(le.classes_))

# ----------------------
# STEP 4: CREATE SLIDING WINDOWS
# ----------------------
def create_windows(X, y, win):
    X_seq, y_seq = [], []
    for i in range(len(X) - win):
        X_seq.append(X[i:i+win])
        y_seq.append(y[i+win-1])  # label at end of window
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq_int = create_windows(X_all, y_int, WINDOW_SIZE)
y_seq_cat = to_categorical(y_seq_int, num_classes)

print("Window Data Shape:", X_seq.shape)

# ----------------------
# STEP 5: TRAIN-TEST SPLIT
# ----------------------
X_train, X_test, y_train_cat, y_test_cat, y_train_int, y_test_int = train_test_split(
    X_seq, y_seq_cat, y_seq_int, test_size=0.2, random_state=RANDOM_STATE, stratify=y_seq_int
)

# ----------------------
# STEP 6: BUILD CNN-LSTM MODEL
# ----------------------
timesteps = X_train.shape[1]
features = X_train.shape[2]

model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(timesteps, features)),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.3),

    LSTM(100),
    Dropout(0.4),

    Dense(50, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

# ----------------------
# STEP 7: TRAIN MODEL
# ----------------------
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train_cat,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop]
)

# ----------------------
# STEP 8: TEST MODEL
# ----------------------
loss, acc = model.evaluate(X_test, y_test_cat)
print("\nTest Accuracy:", acc)

y_pred = np.argmax(model.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test_int, y_pred, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test_int, y_pred))

# ----------------------
# STEP 9: FALL DETECTION FUNCTION
# ----------------------
def predict_fall(window):
    if window.shape != (WINDOW_SIZE, len(SENSOR_COLS)):
        raise ValueError(f"Window shape must be ({WINDOW_SIZE}, {len(SENSOR_COLS)})")

    inp = np.expand_dims(window, axis=0)
    pred = model.predict(inp)[0]
    cls = np.argmax(pred)
    label = le.inverse_transform([cls])[0]

    if "fall" in label.lower() or "fallen" in label.lower():
        print("📢 **FALL DETECTED!**")
    else:
        print("✅ **NOT FALLEN**")

    return label

# ----------------------
# STEP 10: TEST ON A SAMPLE WINDOW
# ----------------------
print("\nTesting prediction on a random window...")

sample = X_test[0]  # real test example
true_label = le.inverse_transform([y_test_int[0]])[0]
pred_label = predict_fall(sample)

print("True Label:", true_label)
print("Predicted Label:", pred_label)


Using Colab cache for faster access to the 'elderly-fall-detection-iot-dataset' dataset.
Dataset downloaded at: /kaggle/input/elderly-fall-detection-iot-dataset
Using data file: /kaggle/input/elderly-fall-detection-iot-dataset/fall_detection.csv
Detected Classes: ['bend', 'fall_backward', 'fall_forward', 'fall_side_left', 'fall_side_right', 'fall_slump', 'lie_down', 'sit', 'stand', 'walk']
Window Data Shape: (24950, 50, 6)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 48, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,032 (285.28 KB)

 Trainable params: 72,904 (284.78 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.2241 - loss: 2.0653 - val_accuracy: 0.2440 - val_loss: 1.9640
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.3080 - loss: 1.8277 - val_accuracy: 0.3347 - val_loss: 1.7851
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.3715 - loss: 1.6871 - val_accuracy: 0.4627 - val_loss: 1.5057
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.4717 - loss: 1.5041 - val_accuracy: 0.5892 - val_loss: 1.2054
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.5814 - loss: 1.2195 - val_accuracy: 0.7430 - val_loss: 0.8034
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.6695 - loss: 0.9796 - val_accuracy: 0.8071 - val_loss: 0.6257
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.7425 - loss: 0.7833 - val_accuracy: 0.8502 - val_loss: 0.4826
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.7879 - loss: 0.6451 - val_